In [214]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow.keras as keras

In [215]:
seqlen=6

In [216]:
scaler=pickle.load(P('问题2-MinMaxScaler-训练数据.csv').open('rb'))
# df=pd.read_csv('问题2-训练数据-归一化.csv',index_col='time', parse_dates=True)
df=pd.read_csv('问题2-训练数据-归一化.csv').drop(columns='time')
model=keras.models.load_model('./problem2_checkpoint.h5')

In [217]:
scaler.scale_, scaler.min_

(array([0.08025682, 0.03125977, 0.0195733 , 0.24509804, 0.0256476 ,
        0.00112134, 0.02069108]),
 array([-7.73675762e-01, -9.28727727e-01, -8.29125073e-01, -4.03137255e+01,
        -6.92485253e-03,  0.00000000e+00,  4.82722946e-01]))

In [218]:
df_test = df.iloc[-seqlen:, :]
test=df_test.values.reshape([1,seqlen,-1])
df_test.shape, test.shape

((6, 7), (1, 6, 7))

In [240]:
pd.period_range?

In [243]:
INDEX=pd.period_range(pd.Period(year=2022,month=4,freq='1M'),pd.Period(year=2023,month=12,freq='1M'))
INDEX

PeriodIndex(['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
             '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
             '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09',
             '2023-10', '2023-11', '2023-12'],
            dtype='period[M]')

In [251]:
def predict(model, df_test: pd.DataFrame, index, seqlen: int):
    n=len(index)
    for i in range(n):
        test=df_test.iloc[-seqlen:, :].values.reshape([1,seqlen,-1])
        pred=model.predict(test)
        df_pred=pd.DataFrame(pred, columns=df.columns)
        df_test = pd.concat([df_test, df_pred])
        
    df_test=df_test.iloc[seqlen:, :]
    df_test.index=index
    
    return df_test

In [252]:
df_pred=predict(model, df_test, INDEX, seqlen=seqlen)
data=scaler.inverse_transform(df_pred.values)
df_pred=pd.DataFrame(data=data, columns=df_pred.columns, index=df_pred.index)
df_pred=df_pred[['10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)', '200cm湿度(kg/m2)']]
df_pred=df_pred.applymap(lambda x: round(x, 2))
df_pred

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),200cm湿度(kg/m2)
2022-04,13.65,36.84,36.74,165.86
2022-05,12.09,33.76,41.19,166.42
2022-06,12.02,34.24,43.53,166.73
2022-07,12.07,34.82,44.54,166.85
2022-08,12.52,36.33,45.29,166.96
2022-09,13.20,38.38,45.69,167.05
2022-10,13.39,39.09,46.16,167.12
2022-11,13.37,39.00,47.68,167.28
2022-12,13.60,39.44,48.32,167.37
2023-01,13.80,39.89,48.74,167.45


In [253]:
df_pred.to_excel('问题2-预测结果.xlsx')